# Checking PoroTomo DAS Channel Geolocation Data

These data are available from https://github.com/feigl/PoroTomo/blob/63b87d358221c0da72b5cc524d5c780c33be1a69/metadata_txt_files/Horizontal_DAS_DTS_UTM_LatLon_Coordinates.xlsx. This spreadsheet contains geolocation for each fibre cable channel as UTM easting and northing, elevation, fiber-optic cable segment azimuth, and latitude and longitude.

The purpose of this notebook is to verify that the PoroTomo geolocation data are in the UTM 11N zone projection coordinates, with the default false earthing and northing values. This information is missing from all the documentation examined so far (it may be somewhere but not obvious where).

In [1]:
import numpy as np
import pandas as pd
import hvplot.pandas
import holoviews as hv
from pyproj import Proj, CRS, Transformer
from pyproj.crs import is_wkt

The original Excel spreadsheet was exported as CSV file:

In [2]:
csv_file = '/Users/ajelenak/Downloads/Horizontal_DAS_DTS_UTM_LatLon_Coordinates.csv'

In [3]:
df = pd.read_csv(csv_file,
                 usecols=(0, 1, 2, 3, 4, 5), 
                 header=0,
                 skiprows=1,
                 names=('channel', 'x_utm', 'y_utm', 'elev', 'lat', 'lon'))

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8721 entries, 0 to 8720
Data columns (total 6 columns):
channel    8721 non-null int64
x_utm      8721 non-null float64
y_utm      8721 non-null float64
elev       8721 non-null float64
lat        8721 non-null float64
lon        8721 non-null float64
dtypes: float64(5), int64(1)
memory usage: 408.9 KB


In [5]:
df.describe()

,channel,x_utm,y_utm,elev,lat,lon
count,8721.000000,8721.000000,8.721000e+03,8721.000000,8721.000000,8721.000000
mean,4340.000000,324774.765212,4.357561e+06,1232.071804,39.349226,-117.638355
std,2517.680182,34982.093274,4.693423e+05,132.955333,4.238210,12.670538
min,-20.000000,0.000000,0.000000e+00,0.000000,0.000000,-119.011341
25%,2160.000000,328308.760000,4.407771e+06,1241.245000,39.802611,-119.005403
50%,4340.000000,328567.510000,4.408092e+06,1248.934000,39.805533,-119.002505
75%,6520.000000,328802.710000,4.408421e+06,1252.410000,39.808549,-118.999903
max,8700.000000,329135.410000,4.408838e+06,1261.511000,39.812319,0.000000


Some channel geolocation is set to zero for all columns which probably indicates they are not really used and so they will be eliminated.

In [6]:
df = df[(df['x_utm'] > 0.) & (df['y_utm'] > 0)].copy()

In [7]:
df.describe()

,channel,x_utm,y_utm,elev,lat,lon
count,8621.000000,8621.000000,8.621000e+03,8621.000000,8621.000000,8621.000000
mean,4340.000000,328542.016867,4.408107e+06,1246.363322,39.805661,-119.002911
std,2488.812669,315.289389,3.873409e+02,8.228047,0.003543,0.003594
min,30.000000,327805.460000,4.407385e+06,1225.596000,39.799102,-119.011341
25%,2185.000000,328319.530000,4.407777e+06,1241.410000,39.802657,-119.005445
50%,4340.000000,328572.760000,4.408095e+06,1249.188000,39.805553,-119.002565
75%,6495.000000,328804.420000,4.408429e+06,1252.456000,39.808617,-118.999964
max,8650.000000,329135.410000,4.408838e+06,1261.511000,39.812319,-118.996102


Plot the channel locations using their UTM coordinates and color each based on its elevation. The elevation data must be relative to the WGS84 geoid since the UTM projection is only 2D.

In [8]:
df.hvplot.scatter(x='x_utm', y='y_utm', c='elev', cmap='viridis', size=1, colorbar=True)

:Scatter   [x_utm]   (y_utm,elev)

Based on the location of the PoroTomo site, it belongs in the UTM 11N zone. Its WKT description is available at http://www.epsg-registry.org/export.htm?wkt=urn:ogc:def:crs:EPSG::32611. Let's check it:

In [9]:
utm11n_wkt = """PROJCRS["WGS 84 / UTM zone 11N",
  BASEGEODCRS["WGS 84",
    DATUM["World Geodetic System 1984",
      ELLIPSOID["WGS 84",6378137,298.257223563,LENGTHUNIT["metre",1.0]]]],
  CONVERSION["UTM zone 11N",
    METHOD["Transverse Mercator",ID["EPSG",9807]],
    PARAMETER["Latitude of natural origin",0,ANGLEUNIT["degree",0.01745329252]],
    PARAMETER["Longitude of natural origin",-117,ANGLEUNIT["degree",0.01745329252]],
    PARAMETER["Scale factor at natural origin",0.9996,SCALEUNIT["unity",1.0]],
    PARAMETER["False easting",500000,LENGTHUNIT["metre",1.0]],
    PARAMETER["False northing",0,LENGTHUNIT["metre",1.0]]],
  CS[cartesian,2],
    AXIS["easting (E)",east,ORDER[1]],
    AXIS["northing (N)",north,ORDER[2]],
    LENGTHUNIT["metre",1.0],
  ID["EPSG",32611]]"""

In [10]:
is_wkt(utm11n_wkt)

True

Let's create a CRS object from this WKT:

In [11]:
utm_crs = CRS(utm11n_wkt)

In [12]:
utm_crs

<CRS: PROJCRS["WGS 84 / UTM zone 11N", BASEGEODCRS["WGS  ...>
Name: WGS 84 / UTM zone 11N
Ellipsoid:
- semi_major_metre: 6378137.00
- semi_minor_metre: 6356752.31
- inverse_flattening: 298.26
Area of Use:
- UNDEFINED
Prime Meridian:
- longitude: 0.0000
- unit_name: degree
- unit_conversion_factor: 0.01745329
Axis Info:
- Easting[E] (east) : (metre)
- Northing[N] (north) : (metre)

In [13]:
utm_crs.to_epsg()

32611

In [14]:
utm_crs.to_proj4()

'+proj=utm +zone=11 +datum=WGS84 +units=m +no_defs +type=crs'

Let's create a projection object from the `utm_crs`.

In [15]:
utm = Proj(utm_crs)

In [16]:
utm.definition_string()

'proj=utm zone=11 datum=WGS84 units=m no_defs ellps=WGS84 towgs84=0,0,0'

In [17]:
utm.to_latlong_def()

'+proj=longlat +datum=WGS84 +no_defs +type=crs'

Convert given lat/lon values to UTM x/y:

In [18]:
proj_x, proj_y = utm(df['lon'].values, df['lat'].values)

In [19]:
type(proj_x)

numpy.ndarray

In [20]:
proj_x.shape

(8621,)

In [21]:
df['proj_x'] = proj_x.tolist()

In [22]:
df['proj_y'] = proj_y.tolist()

In [23]:
df.describe()

,channel,x_utm,y_utm,elev,lat,lon,proj_x,proj_y
count,8621.000000,8621.000000,8.621000e+03,8621.000000,8621.000000,8621.000000,8621.000000,8.621000e+03
mean,4340.000000,328542.016867,4.408107e+06,1246.363322,39.805661,-119.002911,328542.019233,4.408107e+06
std,2488.812669,315.289389,3.873409e+02,8.228047,0.003543,0.003594,315.289343,3.873409e+02
min,30.000000,327805.460000,4.407385e+06,1225.596000,39.799102,-119.011341,327805.463652,4.407385e+06
25%,2185.000000,328319.530000,4.407777e+06,1241.410000,39.802657,-119.005445,328319.532566,4.407777e+06
50%,4340.000000,328572.760000,4.408095e+06,1249.188000,39.805553,-119.002565,328572.758285,4.408095e+06
75%,6495.000000,328804.420000,4.408429e+06,1252.456000,39.808617,-118.999964,328804.421477,4.408429e+06
max,8650.000000,329135.410000,4.408838e+06,1261.511000,39.812319,-118.996102,329135.408767,4.408838e+06


Start checking...

In [24]:
df['rel_err_x_%'] = 100 * (df['proj_x']/df['x_utm'] - 1)
df['rel_err_y_%'] = 100 * (df['proj_y']/df['y_utm'] - 1)

In [25]:
df.describe()

,channel,x_utm,y_utm,elev,lat,lon,proj_x,proj_y,rel_err_x_%,rel_err_y_%
count,8621.000000,8621.000000,8.621000e+03,8621.000000,8621.000000,8621.000000,8621.000000,8.621000e+03,8.621000e+03,8.621000e+03
mean,4340.000000,328542.016867,4.408107e+06,1246.363322,39.805661,-119.002911,328542.019233,4.408107e+06,7.201823e-07,3.455919e-07
std,2488.812669,315.289389,3.873409e+02,8.228047,0.003543,0.003594,315.289343,3.873409e+02,7.444794e-07,7.327726e-09
min,30.000000,327805.460000,4.407385e+06,1225.596000,39.799102,-119.011341,327805.463652,4.407385e+06,-5.915396e-07,3.308577e-07
25%,2185.000000,328319.530000,4.407777e+06,1241.410000,39.802657,-119.005445,328319.532566,4.407777e+06,7.113732e-08,3.394165e-07
50%,4340.000000,328572.760000,4.408095e+06,1249.188000,39.805553,-119.002565,328572.758285,4.408095e+06,7.384640e-07,3.456509e-07
75%,6495.000000,328804.420000,4.408429e+06,1252.456000,39.808617,-118.999964,328804.421477,4.408429e+06,1.345536e-06,3.517533e-07
max,8650.000000,329135.410000,4.408838e+06,1261.511000,39.812319,-118.996102,329135.408767,4.408838e+06,2.021635e-06,3.603764e-07


In [26]:
df.hvplot.scatter(x='x_utm', y='rel_err_x_%', size=2) * hv.HLine(0)

:Overlay
   .Scatter.I :Scatter   [x_utm]   (rel_err_x_%)
   .HLine.I   :HLine   [x,y]

In [27]:
df.hvplot.scatter(x='y_utm', y='rel_err_y_%', size=2)

:Scatter   [y_utm]   (rel_err_y_%)

In [28]:
df.hvplot.hist('rel_err_x_%')

:Histogram   [rel_err_x_%]   (rel_err_x_%_count)

In [29]:
df.hvplot.hist('rel_err_y_%')

:Histogram   [rel_err_y_%]   (rel_err_y_%_count)

Based on the above plots, the spreadsheet geolocation data is verified to be in the UTM 11N zone with the default false easting and northing settings.

## Check Missing Geolocation Values

Missing geolocation in the input spreadsheet is indicated with zero values. Below is what these values means when converted to lat/lon or UTM x/y.

From UTM (x=0, y=0) to lon/lat:

In [30]:
utm(0, 0, inverse=True)

(-121.48874388438719, 0.0)

From (lon=0, lat=0) to UTM x/y:

In [31]:
utm(0, 0)

(9628218.492713422, 19995929.886041995)

What happens if geospatial coordinate values are `NaN`?

In [32]:
utm(np.nan, np.nan)

(1e+30, 1e+30)

In [33]:
utm(np.nan, np.nan, inverse=True)

(1e+30, 1e+30)

The above does not look very helpful... Operations with `NaN`s should yield `NaN`s.